In [1]:
# DELIVERABLE FOR THE PROJECT:
# "KINEMATICS OF MOBILITY"
# David Pastor-Escuredo (LifeD Lab)
# Licencia MIT

In [2]:
#Copyright <2019> <David Pastor Escuredo>

#Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

#The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
import pandas as pd
import numpy as np
import json
from nltk.tokenize import word_tokenize
import re
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import collections
from sklearn.cluster import KMeans
from PIL import Image
import time
from datetime import datetime, timedelta, date
from os import listdir
from os.path import isfile, join
from geopandas import GeoDataFrame
from shapely.geometry import Point
import pickle
import math

%matplotlib inline

def getday(d):
    if d<10:
        ds='0'+str(d)
    else:
        ds=str(d)
    return ds

In [ ]:
#Project onto map

ide=0
desc=['cfbetweenness']
#desc=['ind', 'outd', 'in_degree', 'out_degree','in_eigenvalue','out_eigenvalue','in_betweenness','out_betweenness','dis_betweenness','cfbetweenness','in_closeness','out_closeness','dis_closeness', 'cfcloseness']
ic=0
col=['darkkhaki','firebrick', 'navy', 'olive', 'purple']
stat=['mean','max', 'min', 'median', 'std']
datapath='/home/davidpastor/TEF_mob/'
netpath=datapath+'nets/'
netdescpath=datapath+'nets_desc/'
trajpath=datapath+'trajs/'
trajdesc=datapath+'traj_net/'

limx1=0.0005
limx2=0.002

with open(trajdesc+'TD'+'_'+desc[ide]+'.cnf', 'rb') as fpp:
    VD=pickle.load(fpp) 

with open(trajpath+'HL.tff', 'rb') as fpp:
    HL=pickle.load(fpp) 
    
vector=[]
vectoru=[]
for u in VD:
    vector.append(VD[u][stat[ic]])
    vectoru.append(u)

print(len(vector))
print(len(vectoru))
    
df = pd.DataFrame(list(zip(vectoru, vector)), columns =['user', desc[ide]]) 

su1=df[df[desc[ide]]>=limx1]
su2=su1[su1[desc[ide]]<=limx2]

users=su2['user'].values
userhome=[]
lon=[]
lat=[]

ant_file='antennas/antennas.csv'
sheet=pd.read_csv(ant_file,delimiter=';')
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
sheet['antenna_id']=LAC+Cell

print(len(users))
for u2 in users:
    a=HL[u2]['homeloc']
    userhome.append(a)
    reg=sheet[sheet['antenna_id']==a]
    lon.append(reg['LONGITUD'].values[0])
    lat.append(reg['LATITUD'].values[0])
    
df2 = pd.DataFrame(list(zip(users, userhome, lon, lat)), columns =['user', 'antenna', 'l1', 'l2']) 
print(df2.head())
print(len(df2))
df2.to_csv('TD_'+desc[ide]+'4.csv',index=False)

ucount=[]
lon2=[]
lat2=[]
alist=set(userhome)
print(len(list(alist)))

for a1 in alist:
    au=df2[df2['antenna']==a1]
    count=len(au)
    ucount.append(count)
    reg2=sheet[sheet['antenna_id']==a1]
    lon2.append(reg2['LONGITUD'].values[0])
    lat2.append(reg2['LATITUD'].values[0])

df3 = pd.DataFrame(list(zip(ucount, alist, lon2, lat2)), columns =['nusers', 'antenna', 'l1', 'l2']) 
print(df3.head())
df3.to_csv('TD_'+desc[ide]+'_usercount4.csv',index=False)
print(len(df3))
    

In [19]:
#Project onto map
region='_bogota'
region='_medellin'
th=1

datapath='/home/davidpastor/TEF_mob/'
netdescpath=datapath+'nets_desc/'

ant_file='antennas/antennas'+region+'.csv'
sheet=pd.read_csv(ant_file,delimiter=',')
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
aids=sheet['antenna_id'].values
desc='cfbetweenness'

with open(netdescpath+'NDcfref'+region+'_th'+str(th)+'.cnf', 'rb') as fpp:
    GDcf=pickle.load(fpp)   

v=[]
alist=[]
lon=[]
lat=[]

for a in GDcf:
    if a in aids:
        v.append(GDcf[a][desc])
        alist.append(a)      
        reg=sheet[sheet['antenna_id']==a]
        lon.append(reg['LONGITUD'].values[0])
        lat.append(reg['LATITUD'].values[0])
    
df2 = pd.DataFrame(list(zip(v, alist, lon, lat)), columns =['descriptor', 'antenna', 'l1', 'l2']) 
print(df2.head())
print(len(df2))
df2.to_csv('Maps/geo_'+desc+region+'_th'+str(th)+'.csv',index=False)


   Unnamed: 0  LATITUD  LONGITUD Celda_HEX LAC_HEX antenna_id municipio
0        3096  6.28167  -75.5608      C365    13C7   13C7C365  MEDELLIN
1        3097  6.28167  -75.5608      C366    13C7   13C7C366  MEDELLIN
2        3098  6.28167  -75.5608      C367    13C7   13C7C367  MEDELLIN
3        3099  6.28167  -75.5608      C368    13C7   13C7C368  MEDELLIN
4        3100  6.28167  -75.5608      C369    13C7   13C7C369  MEDELLIN
   descriptor   antenna       l1       l2
0    0.008395  140BD337 -75.5874  6.20919
1    0.006040  1397D17F -75.5672  6.24889
2    0.002373  13D1C946 -75.5731  6.23128
3    0.009226  13D1C942 -75.5731  6.23128
4    0.010109  13D1C4CD -75.5800  6.21139
1122


In [15]:
#Project onto map
region='_bogota'
region='_medellin'
th=1

datapath='/home/davidpastor/TEF_mob/'
netdescpath=datapath+'nets_desc/'

ant_file='antennas/antennas'+region+'.csv'
sheet=pd.read_csv(ant_file,delimiter=',')
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
aids=sheet['antenna_id'].values
desc='in_flow'

with open(netdescpath+'NDedgeref'+region+'_th'+str(th)+'.cnf', 'rb') as fpp:
    GDcf=pickle.load(fpp)   

v=[]
alist=[]
lon=[]
lat=[]

for a in GDcf:
    if a in aids:
        v.append(GDcf[a][desc])
        alist.append(a)      
        reg=sheet[sheet['antenna_id']==a]
        lon.append(reg['LONGITUD'].values[0])
        lat.append(reg['LATITUD'].values[0])
    
df2 = pd.DataFrame(list(zip(v, alist, lon, lat)), columns =['descriptor', 'antenna', 'l1', 'l2']) 
print(df2.head())
print(len(df2))
df2.to_csv('Maps/geo_'+desc+region+'_th'+str(th)+'.csv',index=False)

   Unnamed: 0  LATITUD  LONGITUD Celda_HEX LAC_HEX antenna_id municipio
0        3096  6.28167  -75.5608      C365    13C7   13C7C365  MEDELLIN
1        3097  6.28167  -75.5608      C366    13C7   13C7C366  MEDELLIN
2        3098  6.28167  -75.5608      C367    13C7   13C7C367  MEDELLIN
3        3099  6.28167  -75.5608      C368    13C7   13C7C368  MEDELLIN
4        3100  6.28167  -75.5608      C369    13C7   13C7C369  MEDELLIN
    descriptor   antenna       l1       l2
0   542.516667  13C6C582 -75.5633  6.30556
1   279.533333  13C7C624 -75.5715  6.28811
2   432.933333  13C6CADC -75.5742  6.30028
3   642.933333  13C6CAB5 -75.5567  6.29222
4  1126.583333  13C6C8DD -75.5491  6.30306
1121


In [10]:
#Project onto map
region='_bogota'
#region='_medellin'
th=1

datapath='/home/davidpastor/TEF_mob/'
netdescpath=datapath+'nets_desc/'

ant_file='antennas/antennas'+region+'.csv'
sheet=pd.read_csv(ant_file,delimiter=',')
print(sheet.head())
LAC=sheet['LAC_HEX']
Cell=sheet['Celda_HEX']
aids=sheet['antenna_id'].values
desc='in_closeness'

with open(netdescpath+'NDref'+region+'_th'+str(th)+'.cnf', 'rb') as fpp:
    GDcf=pickle.load(fpp)   

v=[]
alist=[]
lon=[]
lat=[]

for a in GDcf:
    if a in aids:
        v.append(GDcf[a][desc])
        alist.append(a)      
        reg=sheet[sheet['antenna_id']==a]
        lon.append(reg['LONGITUD'].values[0])
        lat.append(reg['LATITUD'].values[0])
    
df2 = pd.DataFrame(list(zip(v, alist, lon, lat)), columns =['descriptor', 'antenna', 'l1', 'l2']) 
print(df2.head())
print(len(df2))
df2.to_csv('Maps/geo_'+desc+region+'_th'+str(th)+'.csv',index=False)

   Unnamed: 0  LATITUD  LONGITUD Celda_HEX LAC_HEX antenna_id    municipio
0           0   4.6092  -74.0718      2711    083F   083F2711  BOGOTÁ D.C.
1           1   4.6092  -74.0718      2712    083F   083F2712  BOGOTÁ D.C.
2           2   4.6092  -74.0718      2713    083F   083F2713  BOGOTÁ D.C.
3           3   4.6092  -74.0718      2714    083F   083F2714  BOGOTÁ D.C.
4           4   4.6092  -74.0718      2715    083F   083F2715  BOGOTÁ D.C.
   descriptor   antenna       l1       l2
0    1.262710  08522784 -74.1084  4.59870
1    1.270302  08522780 -74.1084  4.59870
2    1.262096  08522978 -74.1047  4.58789
3    1.264610  08522781 -74.1084  4.59870
4    1.263978  0853278B -74.0923  4.60748
4742


In [7]:
print(df2.shape)

(1121, 4)
